In [1]:
import numpy as np

In [3]:
import scipy
import scipy.sparse

In [108]:
FOLDER_TEMPLATE = "E:/Championat_com/Lemmatized_Spoiled_Filtered_Numerized/";

In [109]:
from collections import Counter

def create_frequency_matrix(path):
    row, col, data = [], [], []
    with open(path, 'r') as f:
        for i, line in enumerate(f.xreadlines()):
            tokens = map(int, line.split(' '))
            assert tokens[0] == len(tokens) - 1
            words_cnt = Counter(tokens[1:])
            for w, c in words_cnt.iteritems():
                row.append(i)
                col.append(w)
                data.append(c)
    return scipy.sparse.csr_matrix((data, (row, col)))

In [110]:
origin_freq_matrix = create_frequency_matrix(FOLDER_TEMPLATE + 'numerized.dat')

In [571]:
D, W = origin_freq_matrix.shape
T = 10

In [573]:
np.random.seed(42)

phi_matrix = np.random.uniform(size=(T, W)).astype(np.float64)
phi_matrix /= np.sum(phi_matrix, axis=1)[:, np.newaxis]

theta_matrix = np.random.uniform(size=(D, T)).astype(np.float64)
theta_matrix /= np.sum(theta_matrix, axis=1)[:, np.newaxis]

In [686]:
from itertools import izip

def perform_e_step_update(freq_matrix, docptr, phi_matrix, theta_matrix, block_size=1, e_selection_level=0.01):
    D, W = freq_matrix.shape
    T = phi_matrix.shape[0]
    n_wt, n_dt = np.zeros((W, T)), np.zeros((D, T))
    transposed_phi_matrix = np.transpose(phi_matrix)
    
    indices = freq_matrix.indices
    indptr = freq_matrix.indptr
    data = freq_matrix.data
    
    for block_num in xrange((D + block_size - 1) / block_size):
        block_start = block_num * block_size
        block_finish = min(D, block_start + block_size)
        ind_start, ind_finish = indptr[block_start], indptr[block_finish]
        
        datas = data[ind_start:ind_finish]
        words = indices[ind_start:ind_finish]
        docs = docptr[ind_start:ind_finish]
        
        p_dwt = transposed_phi_matrix[words] * theta_matrix[docs, :]
        maximums = np.max(p_dwt, axis=1)
        p_dwt[p_dwt < e_selection_level * maximums[:, np.newaxis]] = 0.
        p_dwt /= (np.sum(p_dwt, axis=1)[:, np.newaxis] + 1e-10)
        p_dwt *= datas[:, np.newaxis]
        
        for doc_num in xrange(block_start, block_finish):
            doc_start, doc_finish = indptr[doc_num], indptr[doc_num + 1]
            doc_p_dwt = p_dwt[(doc_start - ind_start):(doc_finish - ind_start), :]
            n_dt[doc_num, :] += np.sum(doc_p_dwt, axis=0)
            n_wt[indices[doc_start:doc_finish], :] += doc_p_dwt
    return n_wt, n_dt

def trivial_regularization(n_wt, n_dt, phi_matrix, theta_matrix):
    return 0., 0.


def calculate_decorr(phi_matrix):
    aggr_phi = np.sum(phi_matrix, axis=1)
    return np.sum(phi_matrix * (aggr_phi[:, np.newaxis] - phi_matrix))

def create_reg_decorr_naive(tau):
    def fun (n_wt, n_dt, phi_matrix, theta_matrix):
        aggr_phi = np.sum(phi_matrix, axis=1)
        return - tau * np.transpose(phi_matrix * (aggr_phi[:, np.newaxis] - phi_matrix)), -0.1
    return fun

def create_reg_decorr_unbiased(tau):
    def fun (n_wt, n_dt, phi_matrix, theta_matrix):
        tmp_phi =  n_wt / np.sum(n_wt, axis=0)
        aggr_phi = np.sum(tmp_phi, axis=0)
        return - tau * tmp_phi * (aggr_phi[np.newaxis, :] - tmp_phi), -0.1
    return fun

def calculate_likelihood(freq_matrix, docptr, phi_matrix, theta_matrix, block_size=1):
    D, W = freq_matrix.shape
    T = phi_matrix.shape[0]
    transposed_phi_matrix = np.transpose(phi_matrix)
    
    indices = freq_matrix.indices
    indptr = freq_matrix.indptr
    data = freq_matrix.data
    
    res = 0.
    for block_num in xrange((D + block_size - 1) / block_size):
        block_start = block_num * block_size
        block_finish = min(D, block_start + block_size)
        ind_start, ind_finish = indptr[block_start], indptr[block_finish]
        
        datas = data[ind_start:ind_finish]
        words = indices[ind_start:ind_finish]
        docs = docptr[ind_start:ind_finish]
        
        p_dwt = transposed_phi_matrix[words] * theta_matrix[docs, :]
        res += np.sum(np.log(np.sum(p_dwt, axis=1) + 1e-10) * datas)
    
    return res


def create_decorr_logger(tau):
    def fun(freq_matrix, docptr, phi_matrix, theta_matrix):
        likelihood = calculate_likelihood(freq_matrix, docptr, phi_matrix, theta_matrix, 50)
        decorr = calculate_decorr(phi_matrix)
        print 'L', likelihood
        print 'decorr', decorr
        print 'L + tau R', likelihood - tau * decorr
        print 'Zeros', np.sum(phi_matrix <= 1e-20)
    return fun
        

def launch_em(freq_matrix, phi_matrix, theta_matrix, logger, regularization=trivial_regularization, iters_count=100, e_selection_level=0.0001):
    phi_matrix = np.array(phi_matrix)
    theta_matrix = np.array(theta_matrix)
    docptr = []
    indptr = freq_matrix.indptr
    for doc_num in xrange(D):
        docptr.extend([doc_num] * (indptr[doc_num + 1] - indptr[doc_num]))
    docptr = np.array(docptr)
    
    
    for it in xrange(iters_count):
        print it
        n_wt, n_dt = perform_e_step_update(freq_matrix, docptr, phi_matrix, theta_matrix, 50, e_selection_level=e_selection_level)
        r_wt, r_dt = regularization(n_wt, n_dt, phi_matrix, theta_matrix)
        n_wt = np.maximum(n_wt + r_wt, 0)
        n_dt = np.maximum(n_dt + r_dt, 0)
        n_wt /= np.sum(n_wt, axis=0)
        n_dt /= np.sum(n_dt, axis=1)[:, np.newaxis]
        phi_matrix = np.transpose(n_wt)
        theta_matrix = n_dt
        logger(freq_matrix, docptr, phi_matrix, theta_matrix)
        
    return phi_matrix, theta_matrix


In [698]:
%%time
_ = launch_em(
    origin_freq_matrix,
    phi_matrix, theta_matrix,
    create_decorr_logger(1e5),
    create_reg_decorr_unbiased(1e5),
    50,
    e_selection_level=0.5
)

0
L -42532902.8965
decorr 9.98291994257
L + tau R -43531194.8907
Zeros 43867
1
L -42811428.1155
decorr 9.97958949002
L + tau R -43809387.0645
Zeros 74319
2
L -42929201.834
decorr 9.97849571803
L + tau R -43927051.4058
Zeros 90198
3
L -42951047.1659
decorr 9.97826220945
L + tau R -43948873.3869
Zeros 98857
4
L -42945877.2078
decorr 9.97822483339
L + tau R -43943699.6912
Zeros 103805
5
L -42932852.2481
decorr 9.97820330449
L + tau R -43930672.5786
Zeros 106750
6
L -42917261.2069
decorr 9.97816588762
L + tau R -43915077.7956
Zeros 108669
7
L -42900735.3102
decorr 9.97812686409
L + tau R -43898547.9966
Zeros 109949
8
L -42884528.5585
decorr 9.97809804751
L + tau R -43882338.3633
Zeros 110868
9
L -42868913.6814
decorr 9.97807569655
L + tau R -43866721.251
Zeros 111562
10
L -42854135.7527
decorr 9.97806385384
L + tau R -43851942.1381
Zeros 112080
11
L -42840821.992
decorr 9.9780576592
L + tau R -43838627.7579
Zeros 112487
12
L -42829272.0524
decorr 9.97804167507
L + tau R -43827076.2199
Zero

In [713]:
%%time
new_phi, new_theta = launch_em(
    origin_freq_matrix,
    phi_matrix, theta_matrix,
    create_decorr_logger(1e5),
    create_reg_decorr_unbiased(1e5),
    30,
    e_selection_level=0.
)

0
L -42319446.8746
decorr 9.98810687418
L + tau R -43318257.5621
Zeros 0
1
L -42301186.169
decorr 9.98821443901
L + tau R -43300007.6129
Zeros 0
2
L -42283310.9197
decorr 9.98828966701
L + tau R -43282139.8864
Zeros 0
3
L -42263512.1913
decorr 9.98833579793
L + tau R -43262345.7711
Zeros 0
4
L -42239068.6731
decorr 9.98835143128
L + tau R -43237903.8162
Zeros 0
5
L -42206051.8284
decorr 9.9883290578
L + tau R -43204884.7342
Zeros 2
6
L -42158277.5061
decorr 9.98825109464
L + tau R -43157102.6156
Zeros 4
7
L -42086105.7353
decorr 9.98808223309
L + tau R -43084913.9586
Zeros 6
8
L -41976286.4217
decorr 9.98775884382
L + tau R -42975062.306
Zeros 10
9
L -41815847.9308
decorr 9.98718728166
L + tau R -42814566.659
Zeros 10
10
L -41601783.907
decorr 9.98628463234
L + tau R -42600412.3703
Zeros 17
11
L -41350052.4847
decorr 9.98507507356
L + tau R -42348559.992
Zeros 27
12
L -41091100.9597
decorr 9.98373382117
L + tau R -42089474.3419
Zeros 49
13
L -40853500.4684
decorr 9.98248473047
L + tau 

In [714]:
%%time
new_phi2, new_theta2 = launch_em(
    origin_freq_matrix,
    new_phi, new_theta,
    create_decorr_logger(1e5),
    create_reg_decorr_unbiased(1e5),
    10,
    e_selection_level=0.1
)

0
L -39870695.7619
decorr 9.97907745626
L + tau R -40868603.5075
Zeros 110539
1
L -39865721.4909
decorr 9.97897884288
L + tau R -40863619.3752
Zeros 113170
2
L -39859368.3347
decorr 9.97901014146
L + tau R -40857269.3488
Zeros 114765
3
L -39853231.703
decorr 9.97905473692
L + tau R -40851137.1767
Zeros 115965
4
L -39848115.8406
decorr 9.9790907508
L + tau R -40846024.9157
Zeros 116865
5
L -39844097.0088
decorr 9.97911498499
L + tau R -40842008.5073
Zeros 117658
6
L -39841061.5037
decorr 9.97912670499
L + tau R -40838974.1742
Zeros 118309
7
L -39838856.8345
decorr 9.97912859688
L + tau R -40836769.6942
Zeros 118829
8
L -39837295.2399
decorr 9.97912393326
L + tau R -40835207.6332
Zeros 119270
9
L -39836230.473
decorr 9.97911413423
L + tau R -40834141.8865
Zeros 119632
Wall time: 19.1 s


In [735]:
%%time
new_phi3, new_theta3 = launch_em(
    origin_freq_matrix,
    new_phi2, new_theta2,
    create_decorr_logger(1e5),
    create_reg_decorr_unbiased(0),
    10,
    e_selection_level=0.01
)

0
L -39821288.9972
decorr 9.97953564191
L + tau R -40819242.5614
Zeros 119640
1
L -39815417.4664
decorr 9.97964446747
L + tau R -40813381.9131
Zeros 119678
2
L -39812006.7009
decorr 9.97967916758
L + tau R -40809974.6177
Zeros 119746
3
L -39809651.8147
decorr 9.97968973232
L + tau R -40807620.7879
Zeros 119824
4
L -39807884.1876
decorr 9.9796911899
L + tau R -40805853.3066
Zeros 119906
5
L -39806507.0028
decorr 9.9796887416
L + tau R -40804475.8769
Zeros 119997
6
L -39805416.4066
decorr 9.97968436653
L + tau R -40803384.8433
Zeros 120080
7
L -39804540.239
decorr 9.97967912139
L + tau R -40802508.1512
Zeros 120151
8
L -39803825.4689
decorr 9.97967363079
L + tau R -40801792.832
Zeros 120234
9
L -39803234.4893
decorr 9.97966825385
L + tau R -40801201.3146
Zeros 120303
Wall time: 18.8 s


In [744]:
np.sum(new_phi3)

18831.0

In [743]:
m = np.transpose(new_phi3)
m /= np.sum(m, axis=1)[:, np.newaxis]
print np.sum(m > 0)
print np.sum(m >= 0)

68007
188310


In [737]:
num_2_word = {}
with open(FOLDER_TEMPLATE + 'dict.dat', 'r') as f:
    for i, line in enumerate(f.xreadlines()):
        num_2_word[i] = line.strip()

In [738]:
num = 11
print m[num]
print num_2_word[num]

[ 0.          0.37460517  0.05160455  0.          0.05661037  0.
  0.18696495  0.2072494   0.00436107  0.1186045 ]
хозяин


In [739]:
import heapq

In [740]:
for i in xrange(T):
    words = heapq.nlargest(20, range(W), key=lambda num: new_phi3[i, num])
    print i
    for w in words:
        print '\t', num_2_word[w], new_phi3[i, w]

0
	все 0.00957832685686
	очень 0.00896688509526
	они 0.00848492263869
	играть 0.00737234021622
	весь 0.00715664202282
	так 0.00710586889394
	много 0.00695549108909
	тренер 0.00680698769052
	такой 0.00613802093527
	хотеть 0.00602151057319
	сейчас 0.00583043097391
	когда 0.00579242349213
	мой 0.00555959403522
	сказать 0.00552148175642
	знать 0.00544107738631
	чтобы 0.00542013633033
	время 0.00531844941349
	рассказывать 0.00530260256151
	наш 0.0052899989495
	игрок 0.00524000453884
1
	тур 0.00972667407209
	футболист 0.00699637682942
	трансляция 0.00689204593053
	гол 0.00689177223674
	цска 0.00651850644863
	спартак 0.0063127571234
	онлайн 0.00620104127113
	забивать 0.0057761180452
	премьер-лига 0.0053610343152
	встреча 0.00501768706835
	видео 0.00493503067194
	игрок 0.0048316628369
	момент 0.00481567060009
	проводить 0.00457225265421
	зенит 0.00456325076064
	минута 0.00449936793047
	лига 0.00444367139421
	сообщать 0.00423361466308
	мяч 0.00421032774847
	футбол 0.00415759313443
2
	турнир 0.0

In [672]:
%%time
launch_em(origin_freq_matrix, phi_matrix, theta_matrix, create_decorr_logger(1e5), create_reg_decorr_unbiased(1e5), 50)

0
L -42532902.8965
decorr 9.98291994257
L + tau R -43531194.8907
1
L -42811428.1155
decorr 9.97958949002
L + tau R -43809387.0645
2
L -42929201.834
decorr 9.97849571803
L + tau R -43927051.4058
3
L -42951047.1659
decorr 9.97826220945
L + tau R -43948873.3869
4
L -42945877.2078
decorr 9.97822483339
L + tau R -43943699.6912
5
L -42932852.2481
decorr 9.97820330449
L + tau R -43930672.5786
6
L -42917261.2069
decorr 9.97816588762
L + tau R -43915077.7956
7
L -42900735.3102
decorr 9.97812686409
L + tau R -43898547.9966
8
L -42884528.5585
decorr 9.97809804751
L + tau R -43882338.3633
9
L -42868913.6814
decorr 9.97807569655
L + tau R -43866721.251
10
L -42854135.7527
decorr 9.97806385384
L + tau R -43851942.1381
11
L -42840821.992
decorr 9.9780576592
L + tau R -43838627.7579
12
L -42829272.0524
decorr 9.97804167507
L + tau R -43827076.2199
13
L -42819254.794
decorr 9.97801139673
L + tau R -43817055.9337
14
L -42810423.1762
decorr 9.97797166149
L + tau R -43808220.3424
15
L -42802740.9306
decor

In [663]:
%%time
launch_em(origin_freq_matrix, phi_matrix, theta_matrix, create_decorr_logger(1e5), create_reg_decorr_naive(1e5), 50)

0
L -44703155.621
decorr 9.98477366634
L + tau R -45701632.9877
1
L -44392873.5262
decorr 9.9877519217
L + tau R -45391648.7184
2
L -44371442.8485
decorr 9.98719441523
L + tau R -45370162.2901
3
L -44352187.8109
decorr 9.98741005976
L + tau R -45350928.8169
4
L -44327840.773
decorr 9.98737881288
L + tau R -45326578.6543
5
L -44293214.5751
decorr 9.98733562654
L + tau R -45291948.1377
6
L -44239956.5305
decorr 9.98718424113
L + tau R -45238674.9546
7
L -44154766.0403
decorr 9.98686773175
L + tau R -45153452.8135
8
L -44020845.5723
decorr 9.98625785406
L + tau R -45019471.3577
9
L -43827203.0448
decorr 9.9852095901
L + tau R -44825724.0038
10
L -43584495.9021
decorr 9.98370117895
L + tau R -44582866.02
11
L -43327048.129
decorr 9.98196394365
L + tau R -44325244.5233
12
L -43090945.0468
decorr 9.98033560952
L + tau R -44088978.6078
13
L -42895831.9808
decorr 9.97905289336
L + tau R -43893737.2701
14
L -42744546.6119
decorr 9.97818881533
L + tau R -43742365.4934
15
L -42630341.8494
decorr 

In [665]:
%%time
launch_em(origin_freq_matrix, phi_matrix, theta_matrix, create_decorr_logger(1e5), create_reg_decorr_unbiased(1e5), 50)

0
L -42319446.8746
decorr 9.98810687418
L + tau R -43318257.5621
1
L -42301186.169
decorr 9.98821443901
L + tau R -43300007.6129
2
L -42283310.9197
decorr 9.98828966701
L + tau R -43282139.8864
3
L -42263512.1913
decorr 9.98833579793
L + tau R -43262345.7711
4
L -42239068.6731
decorr 9.98835143128
L + tau R -43237903.8162
5
L -42206051.8284
decorr 9.9883290578
L + tau R -43204884.7342
6
L -42158277.5061
decorr 9.98825109464
L + tau R -43157102.6156
7
L -42086105.7353
decorr 9.98808223309
L + tau R -43084913.9586
8
L -41976286.4217
decorr 9.98775884382
L + tau R -42975062.306
9
L -41815847.9308
decorr 9.98718728166
L + tau R -42814566.659
10
L -41601783.907
decorr 9.98628463234
L + tau R -42600412.3703
11
L -41350052.4847
decorr 9.98507507356
L + tau R -42348559.992
12
L -41091100.9597
decorr 9.98373382117
L + tau R -42089474.3419
13
L -40853500.4684
decorr 9.98248473047
L + tau R -41851748.9414
14
L -40653236.2392
decorr 9.9814851235
L + tau R -41651384.7516
15
L -40493767.0027
decorr 